## Amazon Price  Scraper

Creator: Brian Au, HKUST QFIN

Real-time prices is an important indicator of supply-demand dynamics of the market and are likely an important alternative datasource for stock research. This python notebook enables the scraping of the prices and other information of products on Amazon. Coupled with simple analysis, insights can be generated from a proprietary angle on companies.

In [58]:
import requests
import json
import pandas as pd
import numpy as np

#### Search Launch

Enter the product keyword to scrape information about that page.

In [59]:
# Use SSD as the keyword because SSD is a product that shows high price-cyclicality where price-scraping might prove valuable.
product_keyword = "SSD"

In [ ]:
api_key = 'your_api'
fixed_params = {"api_key": api_key, "engine": "amazon", "k": product_keyword}

### Scrape, DF creation and storage

In [61]:
all_results = []
page = 1

while True:
    params = fixed_params.copy()
    params["page"] = page
    search = requests.get("https://serpapi.com/search", params=params)
    response = search.json()

    if "organic_results" not in response or not response["organic_results"]:
        break

    for result in response.get("organic_results", []):
        all_results.append({
            'Product Title': result.get("title"),
            'Price': result.get("price"),
            'Rating': result.get("rating"),
            'Review Count': result.get("reviews"),
            'Link': result.get("link")
        })

    page = page + 1

df = pd.DataFrame(all_results)


Create a new copy dataframe so that we do not annotate the one that is expensive to make (API key usage burns money)

In [77]:
info = df.copy()
info.dropna(inplace=True)

In [78]:
info.head()

,Product Title,Price,Rating,Review Count,Link
0,Crucial BX500 1TB 3D NAND SATA 2.5-Inch Intern...,$100.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-NAND-2-5-...
1,SanDisk 1TB Extreme Portable SSD - Up to 1050M...,$129.95,4.6,86600.0,https://www.amazon.com/SanDisk-1TB-Extreme-Por...
2,"Crucial P310 500GB SSD, PCIe Gen4 NVMe M.2 228...",$61.99,4.8,7000.0,https://www.amazon.com/Crucial-P310-500GB-2280...
3,SanDisk 2TB Extreme Portable SSD - Up to 1050M...,$192.99,4.6,86600.0,https://www.amazon.com/SanDisk-2TB-Extreme-Por...
5,Crucial BX500 4TB 3D NAND SATA 2.5-Inch Intern...,$343.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-2-5-Inch-...


### Analysis

Sort the dataframe by review count to rank the products from the most significant (popular) to the least, so that we can weight or screen the important products for our analysis.

Review count is used because popular and easy-to-be-seen products are intuitively more reviewed, either because it is very good or very bad.

In [79]:
info = info.sort_values(by="Review Count", ascending=False)
info.head()

,Product Title,Price,Rating,Review Count,Link
29,Seagate Portable 4TB External Hard Drive HDD –...,$124.99,4.6,269800.0,https://www.amazon.com/Seagate-Portable-Extern...
0,Crucial BX500 1TB 3D NAND SATA 2.5-Inch Intern...,$100.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-NAND-2-5-...
5,Crucial BX500 4TB 3D NAND SATA 2.5-Inch Intern...,$343.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-2-5-Inch-...
32,Seagate BarraCuda 8 TB Internal Hard Drive HDD...,$169.99,4.6,103300.0,https://www.amazon.com/Seagate-BarraCuda-Inter...
1,SanDisk 1TB Extreme Portable SSD - Up to 1050M...,$129.95,4.6,86600.0,https://www.amazon.com/SanDisk-1TB-Extreme-Por...


In [80]:
info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331 entries, 29 to 102
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Product Title  331 non-null    object 
 1   Price          331 non-null    object 
 2   Rating         331 non-null    float64
 3   Review Count   331 non-null    float64
 4   Link           331 non-null    object 
dtypes: float64(2), object(3)
memory usage: 15.5+ KB


### Significance Index: Logarithmic Transformation
In calculating the market price of the product, more popular products should be associated with a larger weighting. To normalize review counts (combat outlier), we use Index = ln(1+ review count).
Then, index = ln(1+R)/ln(1+Rmax)*100 to standarize.

In [81]:
info["Significance Index"] = np.log(1+info['Review Count'])/np.log(1+info['Review Count'].max())*100
info.head()

,Product Title,Price,Rating,Review Count,Link,Significance Index
29,Seagate Portable 4TB External Hard Drive HDD –...,$124.99,4.6,269800.0,https://www.amazon.com/Seagate-Portable-Extern...,100.000000
0,Crucial BX500 1TB 3D NAND SATA 2.5-Inch Intern...,$100.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-NAND-2-5-...,94.161401
5,Crucial BX500 4TB 3D NAND SATA 2.5-Inch Intern...,$343.99,4.7,130000.0,https://www.amazon.com/Crucial-BX500-2-5-Inch-...,94.161401
32,Seagate BarraCuda 8 TB Internal Hard Drive HDD...,$169.99,4.6,103300.0,https://www.amazon.com/Seagate-BarraCuda-Inter...,92.323040
1,SanDisk 1TB Extreme Portable SSD - Up to 1050M...,$129.95,4.6,86600.0,https://www.amazon.com/SanDisk-1TB-Extreme-Por...,90.912968


### Price DataType Conversion

$xxx.x is a string, limiting mathematical computations, therefore, we change the data in the 'Price' column into a float datatype.

Datatype Error detection

In [82]:
info['Price_char1'] = info['Price'].str[0]
info['Price_char1'].value_counts()

Price_char1
$    331
Name: count, dtype: int64

Second layer detection: See whats wrong with the error data

In [88]:
info.loc[(info['Price_char1']!='$') & (info['Price'].notna()), 'Price']

Series([], Name: Price, dtype: object)

Data Cleaning

In [89]:
info['Price'] = info['Price'].str.replace('$',"").str.replace(',','').str.replace('\t','').astype(float)

In [90]:
weighted_avg_price = ((info['Price'] * info['Significance Index']) / info['Significance Index'].sum()).sum()
print(f'The weighted average price of {product_keyword} is: ${round(weighted_avg_price,2)}.')

The weighted average price of SSD is: $254.73.


### Data Storage

Storing daily prices of Amazon products help in identifying trends

In [93]:
import datetime
today = str(datetime.date.today())

In [95]:
info.to_csv('amazon_scrape_' + product_keyword + '_on_' + today +'.csv')